In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager #installation required
from selenium.webdriver.common.keys import Keys

In [93]:
url = 'https://classic.jobsearch.az/'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
time.sleep(2)
driver.get(url)
time.sleep(5) 

C:\Users\Thosiba\AppData\Local\Temp\ipykernel_4140\2756198728.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [94]:
def navigatePage(driver):
    button = driver.find_element_by_xpath('/html/body/header/div/nav/ul/li[2]/a') 
    button.click()
    time.sleep(3)
    new_button = driver.find_element_by_xpath('//*[@id="all_parts"]/section/div[2]/div/div[5]/a/span[2]')
    new_button.click()
    time.sleep(2)
    return driver.page_source

In [95]:
page_source = navigatePage(driver)

C:\Users\Thosiba\AppData\Local\Temp\ipykernel_4140\455475585.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath('/html/body/header/div/nav/ul/li[2]/a')
C:\Users\Thosiba\AppData\Local\Temp\ipykernel_4140\455475585.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  new_button = driver.find_element_by_xpath('//*[@id="all_parts"]/section/div[2]/div/div[5]/a/span[2]')


In [96]:
driver.execute_script('window.scrollBy(0, document.body.scrollHeight)') #scroll down

In [97]:
ikt_page_source = driver.page_source

In [98]:
soup = BeautifulSoup(ikt_page_source, 'lxml') #parse with BS
job_title = soup.select('h2.vacancies__title  ') 

In [300]:
jTitle = []
for index in range(len(job_title)):
    jTitle.append(job_title[index].text)
jTitle = pd.Series(jTitle)    

In [301]:
dates = soup.select('span.vacancies__dead-line') #date parse

In [302]:
date = []
for dat in dates:
    date.append(dat.text)
date = pd.Series(date)    

In [303]:
df = pd.concat([jTitle, date], keys = ['job_title', 'upload_date'], axis=1)

In [304]:
df['upload_date'] = df['upload_date'].str[0:7].apply(lambda val: val.strip())
df['job_title'] = df['job_title'].apply(lambda val: val.strip())

In [305]:
df.sample(6) 

,job_title,upload_date
17,Sayt üzrə mütəxəssis,15 avq
32,SAP Retail MM Junior Consultant,10 avq
108,İş təminatlı Back-End üzrə bootcamp,20 iyl
74,SQL Developer,29 iyl
92,Senior Business/IT Analyst,25 iyl
104,Backend Developer,21 iyl


In [343]:
df['upload_date'] = '2022-' + df['upload_date'].replace('iyl', '07', regex=True).replace('avq', '08', regex=True). \
replace(' ', '-', regex=True).str[-2:] + '-' + \
df['upload_date'].replace('iyl', '07', regex=True).replace('avq', '08', regex=True). \
replace(' ', '-', regex=True).str[0:2].replace('-', '', regex=True).values

In [344]:
df['upload_date'] = df['upload_date'].astype('datetime64[ns]')

In [345]:
df.sample(4)

,job_title,upload_date
28,İnformasiya təhlükəsizliyi üzrə Baş mütəxəssis,2022-08-12
118,Sistemlərin və şəbəkənin idarə edilməsi şöbəsi...,2022-07-19
105,System administrator,2022-07-21
7,IT System Administrator,2022-08-17


In [346]:
df.duplicated().any()

False

In [347]:
display(df[df['job_title'].str.contains('Data Analitik|Data Scientist|Data|Machine Learning|Data Analyst')].shape[0] \
        , df.shape[0]) #number of data vacancies per month

4

122

In [348]:
df[df['job_title'].str.contains('Data Analitik|Data Scientist|Data|Machine Learning|Data Analyst')].shape[0]/ \
df.shape[0]*100 #pct

3.278688524590164